In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import math
import os 

# Bus Route Combo Analysis

In [4]:
filepath = '../../AT/Strathcona Route Analysis.xlsx'
data = pd.read_excel(filepath, sheet_name='Pull In-Out - ALL', usecols='A:Q', nrows=54)

In [5]:
data.head(5)

,Count,Block #,Pull-Out GARS,Route(s),Local/Commuter,Total Blk Duration,Pull-In GARS,Total Blk Distance,Operating time (h),Charging Time (h),Operating Time (h:mm),Charging time (h:mm),450 BEB,675 BEB,FCEB,NO ZEV,Charging Time
0,1.0,440101.0,05:30:00,401,Commuter,4h40,09:52:00,132.826,4.366667,19.633333,04:22:00,19:38:00,1.0,0.0,0.0,0.0,19.633333
1,2.0,440102.0,06:00:00,401,Commuter,6h40,12:22:00,193.188,6.366667,17.633333,06:22:00,17:38:00,0.0,1.0,0.0,0.0,17.633333
2,3.0,440103.0,12:00:00,401,Commuter,7h40,19:22:00,193.188,7.366667,16.633333,07:22:00,16:38:00,0.0,1.0,0.0,0.0,16.633333
3,4.0,440110.0,15:11:00,401,Commuter,4h59,19:52:00,127.540,4.683333,19.316667,04:41:00,19:19:00,1.0,0.0,0.0,0.0,19.316667
4,5.0,440111.0,15:30:00,401,Commuter,2h56,18:08:00,70.467,2.633333,21.366667,02:38:00,21:22:00,1.0,0.0,0.0,0.0,21.366667


In [6]:
#block_duration_name = ''
#block_distance_name = '' 
# time_in_name = 'Pull-In GARS'
# time_out_name = 'Pull-Out GARS'
# data.rename(columns={time_in_name:'Time-In', time_out_name:'Time-Out'})

In [7]:
data['Time Out'] = pd.to_timedelta(data['Pull-Out GARS'].astype(str))
data['Time In'] = pd.to_timedelta(data['Pull-In GARS'].astype(str))
data['time_out_hrs'] = data['Time Out']/np.timedelta64(1, 'h')
data['time_in_hrs'] = data['Time In']/np.timedelta64(1, 'h')
data.head()
print(data['Total Blk Distance'].nsmallest(5))

36    12.244
42    23.184
47    23.724
45    26.756
32    39.220
Name: Total Blk Distance, dtype: float64


In [8]:
### Adjust variables as needed
BUFFER = 0.5 # hours
MIN_CHARGING_TIME = 10 # hours
MIN_DISTANCE = 0
blocks = []
#MIN_DISTANCE = 100 # kilometers

for i, series in data.iterrows():
  block1 = data.loc[i,'Block #']
  route_type1 = data.loc[i,'Local/Commuter']
  time_out1 = data.loc[i,'time_out_hrs']
  time_in1= data.loc[i,'time_in_hrs']
  distance1 = data.loc[i,'Total Blk Distance']

  for idx, series in data.iterrows():
    if block1 == data.loc[idx,'Block #']:
      next
    else:
      block2 = data.loc[idx,'Block #']
      time_out2 = data.loc[idx,'time_out_hrs']
      time_in2= data.loc[idx,'time_in_hrs']
      distance2 = data.loc[idx,'Total Blk Distance']
      route_type2 = data.loc[idx,'Local/Commuter']
      ### If using min distance use commented out if statement. 
      #if (time_in1 + BUFFER < time_out2) and ((24 + time_out1 ) - time_in2) > MIN_CHARGING_TIME and (distance1 + distance2) > MIN_DISTANCE and route_type1 == route_type2:
      if (time_in1 + BUFFER < time_out2) and ((24 + time_out1 ) - time_in2) > MIN_CHARGING_TIME and route_type1 == route_type2:
        blocks.append([block1, route_type1, block2, route_type2, round(distance1 + distance2, 2), round((24 + time_out1 ) - time_in2, 2),])        

In [9]:
df = pd.DataFrame(blocks, columns=['Block 1',  'Block 1 Route' ,  'Block 2',  'Block 2 Route','Total Distance', 'Charging Time'])
print(df['Block 1'].unique())
print(df['Block 2'].unique())

[440101. 440102. 440301. 440401. 440402. 441101. 441102. 441103. 441104.
 441105. 441106. 441302. 441303. 441402. 441404. 441405. 441406. 443001.]
[440103. 440111. 440410. 440411. 441110. 441111. 441112. 441115. 441308.
 441412. 440110. 441114. 440403. 440310. 441311. 441312. 441313. 444010.
 444110. 444210.]


In [10]:
optimized = []
df_copy = df
while not df.empty:
  max_idx = df['Charging Time'].idxmax()
  block1 = df.loc[max_idx , 'Block 1']
  block2 = df.loc[max_idx , 'Block 2']
  route1 = df.loc[max_idx , 'Block 1 Route']
  route2 = df.loc[max_idx , 'Block 2 Route']
  distance = df.loc[max_idx , 'Total Distance']
  time = df.loc[max_idx , 'Charging Time']

  

  optimized.append([block1, route1, block2, route2, distance, time])
  optimized
  df = df.drop(df[df['Block 1'] == block1].index)
  df = df.drop(df[df['Block 2'] == block2].index)

opt_df = pd.DataFrame(optimized, columns=['Block 1',  'Block 1 Route' ,  'Block 2',  'Block 2 Route', 'Total Distance', 'Charging Time'])

## Results - Optimized Block Combinations

In [11]:
opt_df

,Block 1,Block 1 Route,Block 2,Block 2 Route,Total Distance,Charging Time
0,443001.0,Local,444010.0,Local,35.43,16.27
1,441406.0,Commuter/Local,440310.0,Commuter/Local,109.01,14.50
2,441106.0,Commuter,441115.0,Commuter,198.48,13.55
3,441303.0,Commuter/Local,441313.0,Commuter/Local,128.70,13.45
4,441405.0,Commuter,440111.0,Commuter,109.69,13.45
5,440301.0,Commuter,441110.0,Commuter,264.47,12.87
6,441302.0,Commuter/Local,441311.0,Commuter/Local,101.41,12.80
7,441105.0,Commuter,440410.0,Commuter,162.78,12.55
8,441104.0,Commuter/Local,441312.0,Commuter/Local,135.89,12.05
9,441103.0,Commuter,441111.0,Commuter,297.30,11.45


In [21]:
cwd = '.' + os.getcwd().replace('\\','/')[21:]
FILE_SAVE = './block_combos/full_combo_list.xlsx'
CWD = cwd + FILE_SAVE[1:]

parameters = [BUFFER, MIN_CHARGING_TIME, MIN_DISTANCE, CWD ]
par_table = pd.DataFrame(parameters, index=['Buffer Time (Hrs)', 'Min Charging Time (Hrs)', 'Min Required Combo Distance (km)', 'Current Working Directory'], columns=['Parameters'])
#par_table[0] = 0
par_table

,Parameters
Buffer Time (Hrs),0.5
Min Charging Time (Hrs),10
Min Required Combo Distance (km),0
Current Working Directory,./OneDrive - Zen/Active Customers/ETS/08 - Fle...


## Print

In [23]:
excel_sheets = [par_table, df_copy, opt_df]
sheet_names = ['Parameters','Full Combos List', 'Optimized Combos']
with pd.ExcelWriter(FILE_SAVE) as writer:
        i = 0
        for df in excel_sheets:
            df.to_excel(writer, sheet_name=str(sheet_names[i]))
            i += 1